<a href="https://colab.research.google.com/drive/1j0Wnj0qLUHgjjkNxUk82FmP9aiPnXvoH" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install VectorVault
If you don't have an account, you can sign up for free at [VectorVault.io](https://vectorvault.io)

In [ ]:
!pip install vector-vault 

<br>
<br>

## Get API key to access the Vault Cloud

Enter your name, email, and make a password below to get an API key.

In [ ]:
from vectorvault import register

register(first_name='John', last_name='Smith', email='john@smith.com', password='make_a_password')

<br>

# Connect to VectorVault
In the following code, we import the main Vault class to connect to a vault as well as set our OpenAI API key as an environment varible. We will need that set to process the vectors with `get_vectors` and use the `get_chat()` function later.

In [ ]:
%env OPENAI_API_KEY = YOUR_OPEN_AI_API_KEY

In [ ]:
from vectorvault import Vault
vault = Vault(user='john@smith.com', api_key='YOUR_VECTOR_VAULT_API_KEY', vault='webdump', verbose=True)


<br>

# Download Website and Summarize
We import the download_url function from the `vectorvault` package. Use the defualt url already provided to learn about NASA's Mars mission or enter a another url. The website content is downloaded and saved to a variable called "text_data"

Then we use the `get_chat()` function with `summary=True` to get a summary of the website. This will summarize it no matter how large it is.

In [ ]:
from vectorvault import download_url

text_data = download_url("https://mars.nasa.gov/")

In [ ]:
# Stream a summary of the website with the built-in function `print_stream()`, which returns the full text at the end
summary = vault.print_stream(vault.get_chat_stream(text_data, summary=True))

<br> 

# Save Website Data to Vault

In [ ]:
text_data = summary + '\n' + text_data

# Process the data with add()
vault.add(text_data)

# Get embeddings 
vault.get_vectors()

# Save to Vault
vault.save()

<br> 

# Ask Your Question

In [ ]:
user_input = input("What's your next question?: ")
print("Question:", f"\n{wrap(user_input)}")

# Get response from Language model
answer = vault.print_stream(vault.get_chat_stream(user_input, get_context=True))

<br>

## See where the answer is coming from with `return_context=True`
In this response, we add conversation history to to the "get_chat()" function. We also print the source context that the llm uses for the response. (Since the sources are returned along with the response, it comes in the form of a dictionary)

In [ ]:
user_input2 = input("What's your next question?: ")
print("Question:", f"\n{wrap(user_input2)}")

history = user_input + answer

# metatag is always a list. 'item_id' will pull the item_id metatag from the metadata. metatag_prefixes/suffixes adds text in the print stream to allow you to format the yielded return
answer = vault.print_stream(vault.get_chat_stream(user_input2, history=history, get_context=True, return_context=True, metatag=['item_id'], metatag_prefixes=['\n\nItem: '], metatag_suffixes=['\n']))

In [ ]:
user_input3 = input("What's your next question?: ")
print("Question:", f"\n{wrap(user_input3)}")

history = history + user_input2 + answer

# Get response from Language model
answer = vault.print_stream(vault.get_chat_stream(user_input3, get_context=True))
